In [70]:
import re
import spacy
from spacy.lang.pl.stop_words import STOP_WORDS
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from pandas import read_csv
import pandas as np


In [71]:
# Wczytanie danych
anomaly_opinions = read_csv('csv_data/preprocesed_files/anomaly_opinions.csv', sep=',')
normal_opinions = read_csv('csv_data/preprocesed_files/normal_opinions.csv', sep=',')
anomaly_opinions = anomaly_opinions["content"].values.tolist()
normal_opinions = normal_opinions["content"].values.tolist()

#Podział normalnych opinii na część testową i treningowa, liczba normalnych opinii 12 936,do treningu wykorzystano 90%
normal_opinions_test = normal_opinions[300:400]
normal_opinions = normal_opinions[:200]

print("Wykorzystane normalne opinie: ", len(normal_opinions))
print("Wykorzystane anomalie: ", len(anomaly_opinions))


Wykorzystane normalne opinie:  200
Wykorzystane anomalie:  76


In [72]:
# Inicjalizacja modelu spaCy
nlp = spacy.load('pl_core_news_md')
nlp.pipe_names


['tok2vec',
 'morphologizer',
 'parser',
 'lemmatizer',
 'tagger',
 'attribute_ruler',
 'ner']

In [73]:
#Upraszczanie opini przed dodaniem do DataSet (lower case, tylko litery)
def preprocess_review(review):
    # Usunięcie niepotrzebnych znaków
    cleaned_review = re.sub(r'[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ\s]', '', review)
    # Tokenizacja tekstu
    cleaned_review = cleaned_review.lower()
    return cleaned_review

In [74]:
def tensor_preprocessing(token_list):
    sequence = np.array([token.vector for token in token_list])
    sequence = torch.tensor(sequence, dtype=torch.float32)
    return sequence

In [75]:
def token_preprocessing(opinion):
    token_list = []
    doc = nlp(preprocess_review(opinion))
    for token in doc:
        #TODO: Check if neccesary.
        if not token.is_stop:
            #TODO:
            #token_list.append(token.lemma)
            token_list.append(token)
    return token_list


In [76]:
def preprocessing_pipeline(review):
    token_list = token_preprocessing(review)
    sequence = tensor_preprocessing(token_list)
    # Normalizacja wektorów
    #sequence = nn.functional.normalize(sequence, p=1, dim=1)
    return sequence

In [77]:
# Klasa Dataset do przechowywania danych opinii
class OpinionDataset(Dataset):
    def __init__(self, opinions):
        self.opinions = opinions

    def __len__(self):
        return len(self.opinions)

    #Metoda get_item bierze pojedyńczą opinie wykonuje na niej preprocess_review, następnie wykorzystuję metode nlp z biblioteki Spacy
    #do tokenizacji. Następnie wyciąga wektor word embeddings po tokenizacji i z jego wykorzystaniem tworzy tensor. Ostatnim korkiem
    #jest normalizacja tensora z wykorzystaniem normalizacji norma L1 (Manhattan norm)
    def __getitem__(self, idx):
        return self.opinions[idx]


In [78]:
# Tworzenie datasetów dla danych normalnych treningowych, testowych i opinii o podwójnej jakości
normal_dataset = OpinionDataset(normal_opinions)
anomalous_dataset = OpinionDataset(anomaly_opinions)
normal_test_dataset = OpinionDataset(normal_opinions_test)

# Wyliczenie maksymalnego rozmiaru sekwencji wektorów
max_seq_len1 = max(len(opinion) for opinion in normal_dataset.opinions)
max_seq_len2 = max(len(opinion) for opinion in anomalous_dataset.opinions)
max_seq_len3 = max(len(opinion) for opinion in normal_test_dataset.opinions)
max_seq_len = max(max_seq_len1, max_seq_len2, max_seq_len3)
print("Maksymalna długość opinii: ",max_seq_len)


Maksymalna długość opinii:  1430


In [79]:
# Wyrównanie rozmiarów sekwencji wektorów
def pad_sequence(sequence, max_len):
    if sequence.size(0) == 0:
        return torch.zeros(max_len, 300)
    else:
        padded_seq = torch.zeros(max_len, sequence.size(1))
        padded_seq[:sequence.size(0)] = sequence
        return padded_seq

In [80]:
token_preprocessing("biedronka chodzi po nodze i łaskocze rowerzyste")

[biedronka, chodzi, nodze, łaskocze, rowerzyste]

In [81]:
# Dostosowanie sekwencji wektorów do maksymalnego rozmiaru
normal_dataset = [pad_sequence(preprocessing_pipeline(opinion), max_seq_len) for opinion in normal_dataset]
anomalous_dataset = [pad_sequence(preprocessing_pipeline(opinion), max_seq_len) for opinion in anomalous_dataset]
normal_dataset_test = [pad_sequence(preprocessing_pipeline(opinion), max_seq_len) for opinion in normal_test_dataset]


In [82]:
# Definicja autoenkodera
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2),
            nn.ReLU(),
            nn.Linear(input_dim // 2, input_dim // 4),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(input_dim // 4, input_dim // 2),
            nn.ReLU(),
            nn.Linear(input_dim // 2, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [83]:
# Wielkość wejściowa
input_dim = normal_dataset[0].size(1)

# Inicjalizacja modelu autoenkodera
autoencoder = Autoencoder(input_dim)

# Definicja funkcji straty i optymalizatora
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0001)

# Inicjalizacja pustej macierzy pomyłek
cm = torch.zeros((2, 2), dtype=torch.int)

In [84]:
# Funkcja obliczająca próg na podstawie kwantyla
def calc_threshold(values, quantile):
    return torch.quantile(values, quantile)


# Funkcja trenująca autoenkodera
def train_autoencoder(model, dataloader, criterion, optimizer, quantile, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in dataloader:
            optimizer.zero_grad()
            inputs = batch.float()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Obliczanie macierzy pomyłek
            with torch.no_grad():
                autoencoder.eval()
                mse_loss = nn.MSELoss(reduction='none')
                loss_values = mse_loss(outputs, inputs).mean(dim=(1, 2))
                is_anomalous = torch.where(loss_values > calc_threshold(loss_values, quantile), 1, 0)

                for label in is_anomalous:
                    cm[label][label] += 1

        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    return calc_threshold(loss_values, quantile)

In [85]:
# Tworzenie DataLoader dla danych normalnych
normal_dataloader = DataLoader(normal_dataset, batch_size=32, shuffle=True)

# Ustal wartość kwantyla
quantile = 0.87

# Trenowanie autoenkodera
threshold = train_autoencoder(autoencoder, normal_dataloader, criterion, optimizer, quantile, num_epochs=10)
print("Wyznaczony próg: ", threshold.real)
print("")

# Wyświetlanie macierzy pomyłek
print("Macierz pomyłek:")
print(cm)

Epoch 1/10, Loss: 0.2617
Epoch 2/10, Loss: 0.2601
Epoch 3/10, Loss: 0.2590
Epoch 4/10, Loss: 0.2582
Epoch 5/10, Loss: 0.2570
Epoch 6/10, Loss: 0.2558
Epoch 7/10, Loss: 0.2555
Epoch 8/10, Loss: 0.2537
Epoch 9/10, Loss: 0.2518
Epoch 10/10, Loss: 0.2514
Wyznaczony próg:  tensor(0.2700)

Macierz pomyłek:
tensor([[1690,    0],
        [   0,  310]], dtype=torch.int32)


In [86]:
# Inicializacja macierzy cm
cm = torch.zeros((2, 2), dtype=torch.int)

# Testowanie na opiniach nieświadczących o podwójnej jakości
with torch.no_grad():
    autoencoder.eval()
    for batch in DataLoader(normal_dataset_test, batch_size=1):
        inputs = batch.float()
        outputs = autoencoder(inputs)
        mse_loss = nn.MSELoss(reduction='none')
        loss_values = mse_loss(outputs, inputs).mean(dim=(1, 2))
        is_anomalous = torch.where(loss_values > threshold, 1, 0)
        cm[0][is_anomalous] += 1

In [87]:
# Testowanie na opiniach świadczących o podwójnej jakości
with torch.no_grad():
    autoencoder.eval()
    for batch in DataLoader(anomalous_dataset, batch_size=1):
        inputs = batch.float()
        outputs = autoencoder(inputs)
        mse_loss = nn.MSELoss(reduction='none')
        loss_values = mse_loss(outputs, inputs).mean(dim=(1, 2))
        print(loss_values)
        is_anomalous = torch.where(loss_values > threshold, 1, 0)
        cm[1][is_anomalous] += 1

# Obliczenie sumy wierszy i kolumn macierzy pomyłek
sum_rows = torch.sum(cm, dim=1)
sum_cols = torch.sum(cm, dim=0)

# Obliczenie liczby prawdziwie negatywnych, fałszywie negatywnych, fałszywie pozytywnych i prawdziwie pozytywnych
tn = cm[0][0]
fn = cm[0][1]
fp = cm[1][0]
tp = cm[1][1]

# Wyświetlenie macierzy pomyłek
print("Macierz pomyłek:")
print(cm)
print()

# Wyświetlenie innych metryk oceny
print("Liczba prawdziwie negatywnych (TN):", tn.item())
print("Liczba fałszywie negatywnych (FN):", fn.item())
print("Liczba fałszywie pozytywnych (FP):", fp.item())
print("Liczba prawdziwie pozytywnych (TP):", tp.item())
print()

accuracy = (tn + tp) / (tn + fn + fp + tp)
print("Accuracy:", accuracy.item())

precision = tp / (tp + fp)
print("Precision:", precision.item())

recall = tp / (tp + fn)
print("Recall:", recall.item())

f1_score = 2 * (precision * recall) / (precision + recall)
print("F1-Score:", f1_score.item())

tensor([0.2521])
tensor([0.2420])
tensor([0.2827])
tensor([0.2459])
tensor([0.2481])
tensor([0.2506])
tensor([0.2671])
tensor([0.2653])
tensor([0.2777])
tensor([0.3080])
tensor([0.2444])
tensor([0.2457])
tensor([0.2598])
tensor([0.2485])
tensor([0.3568])
tensor([0.2487])
tensor([0.2485])
tensor([0.2552])
tensor([0.2434])
tensor([0.2593])
tensor([0.2502])
tensor([0.2488])
tensor([0.2441])
tensor([0.2485])
tensor([0.2659])
tensor([0.2434])
tensor([0.2739])
tensor([0.2450])
tensor([0.2536])
tensor([0.2460])
tensor([0.2423])
tensor([0.2515])
tensor([0.2422])
tensor([0.2443])
tensor([0.2514])
tensor([0.2544])
tensor([0.2448])
tensor([0.2556])
tensor([0.2528])
tensor([0.2506])
tensor([0.2864])
tensor([0.2575])
tensor([0.2584])
tensor([0.4626])
tensor([0.2488])
tensor([0.2526])
tensor([0.2466])
tensor([0.2675])
tensor([0.2407])
tensor([0.2528])
tensor([0.2694])
tensor([0.5654])
tensor([0.2653])
tensor([0.2586])
tensor([0.3264])
tensor([0.2882])
tensor([0.2973])
tensor([0.2536])
tensor([0.2420

tensor([[0.4301, 0.4315, 0.4572,  ..., 0.5225, 0.4149, 0.3856],
        [0.5073, 0.4449, 0.4475,  ..., 0.5277, 0.4043, 0.4078],
        [0.4765, 0.4636, 0.4497,  ..., 0.4941, 0.3964, 0.3815],
        [0.4723, 0.4690, 0.4612,  ..., 0.5199, 0.4546, 0.4479],
        [0.3751, 0.3974, 0.3935,  ..., 0.5223, 0.3143, 0.3048]])


In [111]:
import torch
import torch.nn as nn

# Przygotuj tekst testowy
text = "gorszy od niemieckiego"

# Przetwórz tekst przy użyciu preprocessing_pipeline (o ile jest dostępny)


preprocessed_text = pad_sequence(preprocessing_pipeline(text), max_seq_len)


# Przekonwertuj przetworzony tekst na tensor PyTorch
test_data = torch.Tensor(preprocessed_text)

# Przekaż dane testowe przez model
decoded_data = autoencoder(test_data)

# Testowanie na opiniach nieświadczących o podwójnej jakości
with torch.no_grad():
    autoencoder.eval()
    
    inputs = test_data
    outputs = autoencoder(inputs)
    mse_loss = nn.MSELoss(reduction='none')
    loss_values = mse_loss(outputs, inputs).mean()
    print(loss_values)
    is_anomalous = torch.where(loss_values > threshold, 1, 0)
    print(is_anomalous)


tensor(0.2410)
